In [ ]:
import pathlib

import dlsproc.xml

# Aggregates

Directory where the data (*XML* files) are stored

In [ ]:
directory = pathlib.Path.home() / 'xml_parser/sample_files/PlataformasAgregadasSinMenores_202201/'
assert directory.exists()

A (sample) file in that directory

In [ ]:
input_file = directory / 'PlataformasAgregadasSinMenores_20220104_030016_1.atom'
assert input_file.exists()
input_file

PosixPath('/home/manu/xml_parser/sample_files/PlataformasAgregadasSinMenores_202201/PlataformasAgregadasSinMenores_20220104_030016_1.atom')

In [ ]:
!head {input_file} --lines=20

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<feed xmlns="http://www.w3.org/2005/Atom" xmlns:cbc-place-ext="urn:dgpe:names:draft:codice-place-ext:schema:xsd:CommonBasicComponents-2" xmlns:cac-place-ext="urn:dgpe:names:draft:codice-place-ext:schema:xsd:CommonAggregateComponents-2" xmlns:cbc="urn:dgpe:names:draft:codice:schema:xsd:CommonBasicComponents-2" xmlns:cac="urn:dgpe:names:draft:codice:schema:xsd:CommonAggregateComponents-2" xmlns:ns1="urn:oasis:names:specification:ubl:schema:xsd:CommonExtensionComponents-2">
    <author>
        <name>Plataforma de Contratación del Sector Público</name>
        <uri>https://contrataciondelestado.es</uri>
        <email>contrataciondelestado@minhafp.es</email>
    </author>
    <id>https://contrataciondelestado.es/sindicacion/sindicacion_1044/PlataformasAgregadasSinMenores.atom</id>
    <link href="PlataformasAgregadasSinMenores_20220104_030016_1.atom" rel="self"/>
    <link href="PlataformasAgregadasSinMenores.atom" rel="first"/>
    

*Namespaces* in the *XML* file

In [ ]:
# dlsproc.xml.get_namespaces(input_file)

*XML* file is read **and** post-processed.

In [ ]:
df = dlsproc.xml.to_curated_df(input_file)
df.head()

,id,summary,title,updated,ContractFolderStatus - ContractFolderID,ContractFolderStatus - ContractFolderStatusCode,ContractFolderStatus - LocatedContractingParty - BuyerProfileURIID,ContractFolderStatus - LocatedContractingParty - Party - PartyName - Name,ContractFolderStatus - LocatedContractingParty - ParentLocatedParty - PartyName - Name,ContractFolderStatus - ProcurementProject - Name,...,ContractFolderStatus - TechnicalDocumentReference - ID,ContractFolderStatus - TechnicalDocumentReference - Attachment - ExternalReference - URI,ContractFolderStatus - ProcurementProject - PlannedPeriod - StartDate,ContractFolderStatus - ProcurementProject - PlannedPeriod - EndDate,ContractFolderStatus - LocatedContractingParty - Party - PartyIdentification - ID,ContractFolderStatus - LocatedContractingParty - ParentLocatedParty - ParentLocatedParty - PartyName - Name,ContractFolderStatus - TenderingProcess - ParticipationRequestReceptionPeriod - EndDate,ContractFolderStatus - TenderingProcess - ParticipationRequestReceptionPeriod - EndTime,ContractFolderStatus - TenderResult - AwardedTenderedProject - ProcurementProjectLotID,ContractFolderStatus - TenderingProcess - TenderSubmissionDeadlinePeriod
0,https://contrataciondelestado.es/sindicacion/P...,Id licitación: C. 2-2021; Órgano de Contrataci...,L'objecte del contracte és la renovació de tot...,2022-01-03 01:11:41.826000+01:00,C. 2-2021,ADJ,https://contractaciopublica.gencat.cat/ecofin_...,Ajuntament de Sant Ramon,Entitats municipals de Catalunya,L'objecte del contracte és la renovació de tot...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-17 14:00:00
1,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 8128_3/2021; Órgano de Contrata...,Obras de restauración hidromorfológica del río...,2022-01-03 01:00:11.194000+01:00,8128_3/2021,PUB,NaN,Pleno del Ayuntamiento,AYUNTAMIENTO DE MONREAL,Obras de restauración hidromorfológica del río...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-22 23:30:00
2,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 1000_0005-CP01-2021-000063; Órg...,Contrato del servicio de realización de labore...,2022-01-03 01:00:10.399000+01:00,1000_0005-CP01-2021-000063,EV,NaN,El Director General de Comunicación y Relacion...,"Departamento de Presidencia, Igualdad, Función...",Contrato del servicio de realización de labore...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 1379/2020 4738; Órgano de Contr...,Obres de renovació de l'enllumenat públic a la...,2022-01-03 00:11:40.740000+01:00,1379/2020 4738,EV,https://contractaciopublica.gencat.cat/ecofin_...,Ajuntament de Canet de Mar,Entitats municipals de Catalunya,Obres de renovació de l'enllumenat públic a la...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-02 23:59:00
4,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 2021-44; Órgano de Contratación...,Subministre i la instal·lació fotovoltaica en ...,2022-01-03 00:11:40.696000+01:00,2021-44,EV,https://contractaciopublica.gencat.cat/ecofin_...,Ajuntament de Valls,Entitats municipals de Catalunya,Subministre i la instal·lació fotovoltaica en ...,...,Enllac plec clausules tecniques.doc,https://contractaciopublica.gencat.cat/ecofin_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-02 23:59:00


Most types are `object` since there are missing values everywhere.

In [ ]:
df.dtypes[:4]

id                                       object
summary                                  object
title                                    object
updated    datetime64[ns, pytz.FixedOffset(60)]
dtype: object

It should work for any file

In [ ]:
another_input_file = directory / 'PlataformasAgregadasSinMenores.atom'
assert another_input_file.exists()
another_input_file

PosixPath('/home/manu/xml_parser/sample_files/PlataformasAgregadasSinMenores_202201/PlataformasAgregadasSinMenores.atom')

*XML* file is read **and** post-processed.

In [ ]:
df = dlsproc.xml.to_curated_df(another_input_file)
df.head()

,id,summary,title,updated,ContractFolderStatus - ContractFolderID,ContractFolderStatus - ContractFolderStatusCode,ContractFolderStatus - LocatedContractingParty - Party - PartyName - Name,ContractFolderStatus - LocatedContractingParty - ParentLocatedParty - PartyName - Name,ContractFolderStatus - ProcurementProject - Name,ContractFolderStatus - ProcurementProject - TypeCode,...,ContractFolderStatus - TenderingProcess - ParticipationRequestReceptionPeriod - EndTime,ContractFolderStatus - TenderResult - ResultCode,ContractFolderStatus - TenderResult - ReceivedTenderQuantity,ContractFolderStatus - TenderResult - WinningParty - PartyIdentification - ID,ContractFolderStatus - TenderResult - WinningParty - PartyName - Name,ContractFolderStatus - TenderResult - AwardedTenderedProject - LegalMonetaryTotal - TaxExclusiveAmount,ContractFolderStatus - LocatedContractingParty - Party - PartyIdentification - ID,ContractFolderStatus - TenderResult - AwardedTenderedProject - ProcurementProjectLotID,ContractFolderStatus - LocatedContractingParty - ParentLocatedParty - ParentLocatedParty - PartyName - Name,ContractFolderStatus - TenderingProcess - TenderSubmissionDeadlinePeriod
0,https://contrataciondelestado.es/sindicacion/P...,Id licitación: CONTR 2021 0000416750; Órgano d...,obras sustitución equipos climatización edif. ...,2022-02-01 01:11:54.877000+01:00,CONTR 2021 0000416750,EV,Dirección Provincial del Servicio Andaluz de E...,Junta de Andalucía,obras sustitución equipos climatización edif. ...,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-31 23:59:00
1,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 8023_1/2022; Órgano de Contrata...,obras forestales,2022-02-01 01:00:15.210000+01:00,8023_1/2022,PUB,Ayuntamiento Anue,AYUNTAMIENTO DE ANUE,obras forestales,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-02-15 23:30:00
2,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 1009_00001119-2021-0031; Órgano...,Proyecto de recuperación del espacio fluvial d...,2022-02-01 01:00:14.306000+01:00,1009_00001119-2021-0031,EV,Dirección General de Medio Ambiente,Desarrollo Rural y Medio Ambiente,Proyecto de recuperación del espacio fluvial d...,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 3064_23/2021; Órgano de Contrat...,Contrato del Servicio para el desarrollo de la...,2022-02-01 01:00:13.239000+01:00,3064_23/2021,EV,Junta de Gobierno Local,Ayuntamiento de Zizur Mayor,Contrato del Servicio para el desarrollo de la...,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 451/2021; Órgano de Contratació...,Projecte d'obres de diverses àrees de gossos a...,2022-02-01 00:11:59.636000+01:00,451/2021,EV,Ajuntament de Sabadell,Entitats municipals de Catalunya,Projecte d'obres de diverses àrees de gossos a...,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-31 23:59:00
